# Pandas: Heroes of Pymoli

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
total_players = purchase_data['SN'].unique()
datacount = purchase_data['SN'].count()
print(f'Unique players {len(total_players)} out of {datacount} entries')

Unique players 576 out of 780 entries


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
unique_items = purchase_data['Item Name'].unique()
print(f'Number of Unique Items is {len(unique_items)}')

Number of Unique Items is 179


In [5]:
average_price = purchase_data['Price'].mean()
total_revenue = purchase_data['Price'].sum()
print(f'Average price is ${average_price:0.2f} for a total revenue of ${total_revenue:.2f}')

Average price is $3.05 for a total revenue of $2379.77


In [6]:
total_df = pd.DataFrame({'Unique Items' : [f'{len(unique_items)}'],
                        'Average Price' : [f'${average_price:0.2f}'],
                         'Purchases'    : [f'{datacount}'],
                        'Total Revenue' : [f'${total_revenue:.2f}']})
total_df.index = ['Summary']

In [7]:
print("Summary: Purchase Analysis")
total_df

Summary: Purchase Analysis


,Unique Items,Average Price,Purchases,Total Revenue
Summary,179,$3.05,780,$2379.77


## Gender Demographics

In [8]:
byID = purchase_data.groupby("SN")
player_count = len(byID)
demographics = purchase_data.groupby(['SN', 'Gender']).size().unstack().count()
df = pd.DataFrame(demographics)
df.columns = ['Count']
df['Percent'] = df['Count'] / player_count * 100
df['Percent'] = df['Percent'].map('{:0.2f}%'.format)

In [9]:
print("Results: Gender Demographics")
df

Results: Gender Demographics


,Count,Percent
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [10]:
purch_count = purchase_data.groupby(['SN', 'Gender']) ['Price'].agg(['sum','count'])
df = purch_count.reset_index()


In [11]:
byGender = df.groupby('Gender').agg({'sum': ['count', 'sum', 'mean'],
                         'count': ['mean','sum']})
byGender.columns = ['Number of Buyers', 
                    'Purchase Total', 
                    'Average Purchase per Person', 
                    'Avg. Purchases Per Person',
                   'Total Items']


byGender['Average Purchase Price'] = byGender['Purchase Total'] / byGender['Total Items']

byGender['Average Purchase per Person'] = byGender['Average Purchase per Person'].map('${:0.2f}'.format)
byGender['Average Purchase Price'] = byGender['Average Purchase Price'].map('${:0.2f}'.format)
byGender['Purchase Total'] = byGender['Purchase Total'].map('${:0.2f}'.format)

In [12]:
byGender_df = byGender[['Number of Buyers','Total Items', 'Average Purchase Price','Purchase Total', 'Average Purchase per Person']]

In [13]:
byGender_df

,Number of Buyers,Total Items,Average Purchase Price,Purchase Total,Average Purchase per Person
Gender,,,,,
Female,81,113,$3.20,$361.94,$4.47
Male,484,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,11,15,$3.35,$50.19,$4.56


## Age Demographics

In [14]:
age_max = max(purchase_data['Age'])
age_min = min(purchase_data['Age'])

In [15]:
ages = [a for a in range(10,age_max,4)]
ages.insert(0, 0)
ages.append(100)
age_labels = [str(a) for a in ages]
labels = []
for idx, alab in enumerate(age_labels):
    if idx == 0:
        labels.append(f'< {age_labels[idx+1]}')
    elif idx == len(ages) - 2:
        
        labels.append(f'> {age_labels[idx]}')
        break
    else:
        labels.append(f'{age_labels[idx]}-{age_labels[idx+1]}')
        

In [16]:
by_age_df = purchase_data
by_age_df['Age Category'] = pd.cut(purchase_data['Age'], ages, labels = labels)

In [17]:
by_age_df_2 = by_age_df.groupby(['SN','Age Category'])['Price'].agg(['sum','count'])
df = by_age_df_2.reset_index()
df = df[df['count']>0]
df['person'] = 1
df = df.groupby('Age Category').agg({'count':'count'}).reset_index()
df['Percent'] = df['count']/ player_count * 100

df['Percent'] = df['Percent'].map('{:0.2f}%'.format)
df = df.rename(columns={'count':'Number of Buyers'})

In [18]:
df

,Age Category,Number of Buyers,Percent
0,< 10,24,4.17%
1,10-14,15,2.60%
2,14-18,90,15.62%
3,18-22,178,30.90%
4,22-26,151,26.22%
5,26-30,48,8.33%
6,30-34,27,4.69%
7,34-38,25,4.34%
8,38-42,14,2.43%
9,> 42,4,0.69%


## Purchasing Analysis (Age)

In [19]:
age_purch_df = by_age_df.groupby(['SN', 'Age Category']) ['Price'].agg(['sum','count'])
df = age_purch_df.reset_index()
df = df[df['count']>0]

byAge = df.groupby('Age Category').agg({'sum': ['count', 'sum', 'mean'],
                        'count': ['mean','sum']})
byAge.columns = ['Player Count', 
                    'Purchase Total', 
                    'Average Purchase per Person', 
                    'Avg. Purchases Per Person',
                   'Total Items']

byAge['Average Purchase Price'] = byAge['Purchase Total'] / byAge['Total Items']

byAge = byAge[['Player Count','Total Items', 'Average Purchase Price','Purchase Total', 'Average Purchase per Person']]
byAge['Average Purchase Price'] = byAge['Average Purchase Price'].map('${:0.2f}'.format)
byAge['Average Purchase per Person'] = byAge['Average Purchase per Person'].map('${:0.2f}'.format)
byAge['Purchase Total'] = byAge['Purchase Total'].map('${:0.2f}'.format)

In [20]:
byAge

,Player Count,Total Items,Average Purchase Price,Purchase Total,Average Purchase per Person
Age Category,,,,,
< 10,24,32,$3.41,$108.96,$4.54
10-14,15,19,$2.68,$50.95,$3.40
14-18,90,113,$3.03,$342.91,$3.81
18-22,178,254,$3.04,$771.89,$4.34
22-26,151,207,$3.06,$634.24,$4.20
26-30,48,63,$2.88,$181.23,$3.78
30-34,27,38,$2.73,$103.68,$3.84
34-38,25,35,$3.55,$124.35,$4.97
38-42,14,15,$3.37,$50.50,$3.61


## Top Spenders

In [21]:
top_df = purchase_data.groupby(['SN'])['Price'].agg(['count', 'mean', 'sum'])
top_df = top_df.reset_index()
top_df.rename(columns={'SN' : 'Player ID',
                         'count' : 'Quantity',
                       'mean': 'Average',
                        'sum': 'Purchase Total'
                        }, inplace = True)

top_df.sort_values('Purchase Total',ascending = False, inplace = True)
top_df = top_df.set_index('Player ID')
top_df['Purchase Total'] = top_df['Purchase Total'].map('${:0.2f}'.format)
top_df['Average'] = top_df['Average'].map('${:0.2f}'.format)


top_df.iloc[:5,:]

,Quantity,Average,Purchase Total
Player ID,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [22]:
items_df = purchase_data[['Item ID', 'Item Name', 'Price']]

lookup = items_df.drop_duplicates(subset=['Item ID'])

df = items_df.groupby('Item ID',as_index=False,group_keys=False)['Price'].agg(['sum', 'count'])
df1 = df.reset_index()
df2_ = pd.merge(df1, lookup, on='Item ID')
df2 = df2_.copy()


df2['Price'] = df2['Price'].map('${:0.2f}'.format)
df2['sum'] = df2['sum'].map('${:0.2f}'.format)
df3 = df2.sort_values('count', ascending = False)

df3 = df3[['Item ID','Item Name', 'count', 'Price', 'sum']]
df3 = df3.rename(columns = {'Item ID': 'ID',
                      'Item Name':'Name', 
                      'count': 'Quantity', 
                      'Price': 'Unit Price', 
                      'sum': 'Total'})
df3=df3.set_index('ID')

In [23]:
df3

,Name,Quantity,Unit Price,Total
ID,,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
...,...,...,...,...
42,The Decapitator,1,$1.75,$1.75
51,Endbringer,1,$4.66,$4.66
118,"Ghost Reaver, Longsword of Magic",1,$2.17,$2.17


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [24]:
df4 = df2_.copy()

df4 = df4[['Item ID','Item Name', 'count', 'Price', 'sum']]

df5 = df4.sort_values('sum', ascending =False)

df5['Price'] = df5['Price'].map('${:0.2f}'.format)
df5['sum'] = df5['sum'].map('${:0.2f}'.format)

df6 = df5.rename(columns = {'Item ID': 'ID',
                      'Item Name':'Name', 
                      'count': 'Quantity', 
                      'Price': 'Unit Price', 
                      'sum': 'Total'})
df6 = df6.set_index("ID")

In [25]:
df6

,Name,Quantity,Unit Price,Total
ID,,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
...,...,...,...,...
28,"Flux, Destroyer of Due Diligence",2,$1.06,$2.12
125,Whistling Mithril Warblade,2,$1.00,$2.00
126,Exiled Mithril Longsword,1,$2.00,$2.00


## Summary

The world of _Heroes of Pymoli_ has a done well this quarter. It has sold a total of 780 of items for a revenue of $2379.77. While some females take part in our game, 84\% of our customers are male with most of them between 18 and 22 years old. 

This past quarter our <u>best</u> selling item was __Final Critic__ at 13 units for a revenue of $59.99.

Let's keep up the good work!

___The Boss___